# 필요한 라이브러리 import

In [54]:
import os
import json
import numpy as np
import pandas as pd
from scipy import sparse as spr
import configparser
import pymysql
from itertools import chain
from collections import defaultdict,Counter
import hnswlib
import warnings
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import NMF
from itertools import repeat
from sklearn.preprocessing import MinMaxScaler
import random
from scipy.stats import skew,kurtosis
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [55]:
# 노래 데이터
with open('./data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
song_df = pd.DataFrame.from_dict(song_dict)

In [56]:
#장르 데이터
with open('./data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

In [57]:
# 플레이리스트 데이터
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)

# 태그-id간 동기화를 위한 데이터베이스 연결

## 데이터베이스 연결 정보

In [58]:
# config parser 객체 생성
config = configparser.ConfigParser()

# parser객체로 config.ini 파일 읽기
config.read('config.ini')

# 연결 config 변수저장

user = config['DEFAULT']['ADMIN_USER_NAME']
passwd = config['DEFAULT']['ADMIN_PASSWORD']
host = config['DEFAULT']['RDS_ENDPOINT']
port = config['DEFAULT']['PORT']
database = config['DEFAULT']['DEFAULT_DATABASE']

## 데이터베이스 연결 및 쿼리 실행

In [59]:
# 데이터 베이스 연결
melon_recom_db = pymysql.connect(
    host = host,
    user = user,
    password = passwd,
    database = database,
    port = int(port),
    cursorclass = pymysql.cursors.DictCursor
)

# 커서 생성 후 쿼리문 수행
melon_cursor = melon_recom_db.cursor()
melon_cursor.execute('select tag_id,tag from tag')
tag_db_list=melon_cursor.fetchall()

## tag와 tag_id 간 딕셔너리 만들기 

In [60]:
# tag를 id로
tag_to_id = {}

# id를 tag로
id_to_tag = {}

for tag_db in tag_db_list:
    tag_to_id[tag_db['tag']] = tag_db['tag_id']
    
for tag_db in tag_db_list:
    id_to_tag[tag_db['tag_id']] = tag_db['tag']

# 태그 전처리

## 태그 사용 빈도수 도출

In [61]:
# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = train_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

In [62]:
# 태그 빈도의 분포를 알아보자
tag_freq = pd.DataFrame().from_dict(tags_frequency,orient="index")
tag_freq.columns=['빈도']
tag_freq.describe().T

,count,mean,std,min,25%,50%,75%,max
빈도,29160.0,16.335082,247.011075,1.0,1.0,1.0,3.0,16465.0


## 태그 사용 빈도 수의 평균 값을 기준으로 태그 전처리

In [63]:
# 빈도수로 태그를 filter

def filter_func(x):
    temp = []
    for tag in x:
        if tags_frequency[tag] >=16:
            temp.append(tag)
        else:
            pass
    return temp
            
train_df['tags'] = train_df['tags'].map(filter_func)

In [64]:
# filter된 태그의 id만 남기기
train_df['tag_ids'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])
train_df.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,tag_ids
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304]


## 컬럼명 한글화 및 플레이리스트 당 태그의 수 컬럼 생성

In [65]:
train_df.columns=['태그','플리아이디','플리제목','노래들','좋아요수','변경일자','태그아이디']
train_df['태그수'] = train_df['태그'].map(len)

# 플레이리스트의 태그로 예측에 사용 될 태그와 검증에 사용 될 태그로 나누기

In [66]:
# 태그가 3개인거 부터 train에 쓸거임
origin_tags = train_df[train_df['태그수']>3]['태그'].tolist()

train_tags = []
test_tags = []

# 3:7비율로 나누기
for tags in origin_tags:
    tag_3p = len(tags)//3
    train_tag = random.sample(tags, tag_3p)
    test_tag = list(set(tags)-set(train_tag))
    train_tags.append(train_tag)
    test_tags.append(test_tag)
    
cal_tag_hit_df = pd.DataFrame(columns=['예측용태그','검증용태그','svd_예측결과','svd_히트'])

cal_tag_hit_df['예측용태그'] = train_tags
cal_tag_hit_df['검증용태그'] = test_tags
cal_tag_hit_df.head(1)

,예측용태그,검증용태그,svd_예측결과,svd_히트
0,"[눈오는날, 따듯한]","[연말, 크리스마스, 캐럴, 겨울왕국, 분위기, 겨울노래]",NaN,NaN


# 세부장르 전처리 및 플레이리스트에 세부장르 정보 넣기

In [67]:
# 노래 데이터 프레임 컬럼명 한글화
song_df.colums=['세부장르','발매일','앨범명','앨범ID','가수ID','노래명','대장르','가수명','노래ID']

## 세부 장르 이름에 대장르의 이름 넣기

In [68]:
genre_dict['GN9000'] = '기타장르'

genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

genre_detail_dict = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        genre_detail_dict[k] = new_value
        
genre_big_dict = {}

for k,v in genre_big.items():
    genre_big_dict[k+'00'] = v

## 세부장르ID와 CODE간 딕셔너리 생성

In [69]:
genre_detail_id_to_code = {}
genre_detail_code_to_id = {}

for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_id_to_code[i] = v
    
for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_code_to_id[v] = i

## 노래ID와 세부장르ID간 딕셔너리 생성

In [70]:
# 노래의 ID를 KEY 세부장르의 ID리스트를 ITEM으로 하는 딕셔너리 생성 

song_genre_detail_dict = defaultdict(list)

for codes,id in zip(song_df['song_gn_dtl_gnr_basket'].tolist(),song_df['id'].tolist()):
    for code in codes:
        song_genre_detail_dict[id].append(genre_detail_code_to_id[code])

## 플레이리스트 각 노래마다 달린 세부장르의 ID를 전부 플레이리스트에 추가

In [71]:
# 플레이리스트의 모든 노래의 세부장르를 모아서 새로운 컬럼으로 생성

def fetcher(x):
    temp = []
    for song in x:
        genre_ids = song_genre_detail_dict[song]
        for id in genre_ids:
            temp.append(id)
    return temp

train_df['세부장르'] = train_df['노래들'].map(fetcher)
train_df.head(1)

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,태그아이디,태그수,세부장르
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[25304],1,"[97, 96, 48, 49, 56, 67, 60, 56, 68, 48, 49, 5..."


## 태그ID와 세부장르ID들 간 딕셔너리 생성

In [72]:
# {tag_Id:genre_detail_ids}

tag_id_to_genre_detail_ids = defaultdict(list)

for ids,genres in zip(train_df['태그아이디'].tolist(),train_df['세부장르'].tolist()):
    for id in ids:
        tag_id_to_genre_detail_ids[id].extend(genres)

# 태그 임베딩

## 세부장르의 빈도를 Min-Max Scaler로 0~1사이의 값으로 변환

In [73]:
# 장르의 빈도수를 카운트해서 value를 바꿈

for k,v in tag_id_to_genre_detail_ids.items():
    tag_id_to_genre_detail_ids[k] = dict(Counter(v))

In [74]:
for k,v in tag_id_to_genre_detail_ids.items():
    if len(v.values()) == 0:
        print(k)

In [75]:
# 딕셔너리의 빈도를 min_max_scaling하고 1을 더함 -> 0이 되는것은 싫다!
for k,value_dict in tag_id_to_genre_detail_ids.items():
    max_val = np.max(list(value_dict.values()))
    min_val = np.min(list(value_dict.values()))
    for key,value in value_dict.items():
        value_dict[key] = np.round((value-min_val)/(max_val-min_val),3)

## 만든 딕셔너리를 이용해서 csr_matrix 생성

In [76]:
# 행,열,데이터 list를 생성
row = []
col = []
dat = []

for k,v in tag_id_to_genre_detail_ids.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

In [77]:
# 29160의 태그의 가짓수, 224는 세부장르의 가짓수
A = spr.csr_matrix((dat, (row, col)), shape=(29160, 224))

# 행렬분해 수행을 위해 자료형 변경
A = A.astype(float)

# SVD로 태그 임베딩

## 태그-세부장르 행렬의 SVD 행렬분해 수행 

In [78]:
# svd행렬분해 100개의 sigular vector사용
u,s,vt = spr.linalg.svds(A,k=223)
svd_vectors = u
data_len,dim = svd_vectors.shape

# svd tag의 벡터들을 저장
with open('./data/svd_tag_vectors.pickle', 'wb') as f:
    pickle.dump(svd_vectors,f)

## cosine유사도를 이용해서 KNN 모델 생성

In [79]:
# 100차원으로 index 생성 및 초기화
svd_p = hnswlib.Index(space='cosine', dim=dim)  
svd_p.init_index(max_elements=data_len, ef_construction=1000, M=300)

# 짐재행렬 추가
svd_p.add_items(svd_vectors,np.arange(data_len))

#svd knn모델 저장
with open('./data/svd_knn_model.pickle', 'wb') as f:
    pickle.dump(svd_p,f)

# 태그 예측

## 예측 수행 및 수행 결과 저장

In [80]:
# svd_knn_model로 태그를 예측하여 예측된 결과의 list를 저장하는 함수

def tag_svd(x):
    target_ids = [tag_to_id[t] for t in x]
    
    vectors = np.zeros((100,1))
    
    for id in target_ids:
        vectors = vectors+svd_vectors[id]
    
    labels, distances = svd_p.knn_query(vectors/len(target_ids), k = 10+len(x))

    ids = [label for label in labels[0]]
   
    return list(set([id_to_tag[tag] for tag in ids])-set(x))

In [81]:
# 예측수행
cal_tag_hit_df['svd_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_svd)

## 평가 지표

In [82]:
val_tag = cal_tag_hit_df['검증용태그'].tolist()
svd_tag = cal_tag_hit_df['svd_예측결과'].tolist()

svd_hit = []

for val,svd in zip(val_tag,svd_tag):
    if len(val) == len(set(val)-set(svd)):
        svd_hit.append(0)
    else:
        svd_hit.append(1)
        
cal_tag_hit_df['svd_히트'] = svd_hit

In [85]:
svd_inter = []

for val,svd in zip(val_tag,svd_tag):
    svd_inter.append(len(set(val) - (set(val)-set(svd))))
        
cal_tag_hit_df['svd_교집합'] = svd_inter

In [83]:
sum(svd_hit)/len(svd_hit)

0.4749931550606918

In [86]:
cal_tag_hit_df.head(30)

,예측용태그,검증용태그,svd_예측결과,svd_히트,svd_교집합
0,"[눈오는날, 따듯한]","[연말, 크리스마스, 캐럴, 겨울왕국, 분위기, 겨울노래]","[눈, 감성발라드, 첫눈, 크리스마스, 겨울, 잔잔한, 혼자, 추운날, Winter...",1,1
1,"[드라이브, 기분전환, 팝]","[운동, 힐링, 트로피컬하우스, Pop, 일렉, 2017, 트렌드]","[운동, 팝송모음, 스트레스, 팝송추천, 매장음악, 추천곡, 산책, popsong,...",1,2
2,"[고백, 사랑]","[취향저격, 이별, 짝사랑, 슬픔]","[데이트, 사랑노래, 달달한노래, 달달, 연애세포, 연인, 두근두근, 연애, 커플, 썸]",0,0
3,"[댄스, 락]","[포크, 메탈, 일렉트로니카, 인디]","[락음악, 밴드음악, 스트레스, Rock, Coldplay, 활력, 2000_10,...",0,0
4,"[락, 록]","[Metal, M에센셜, Rock, 메탈]","[락음악, 밴드음악, Rock, Coldplay, 2000_10, 밴드, 모던락, ...",1,1
5,"[느낌있는, 밤, 새벽]","[감각적인, RnB, 그루브한, 힙합, 드라이브, 국내]","[추천노래, 띵곡, 매장음악, 산책, 취향저격, 잔잔한, 그루브, 분위기, 비오는날...",0,0
6,[질리지않는],"[나만알고싶은, 감성, Pop]","[팝송추천, 퓨처베이스, AlanWalker, 디즈니OST, popsong, 겨울왕...",0,0
7,[새벽감성],"[혼자, 고민, 조용히]","[비, 퇴근길, 쓸쓸한, 상처, 새벽에듣기좋은노래, 쓸쓸, 잔잔한, 울고싶을때, 밤...",0,0
8,"[잔잔한, 편안한]","[밤, 감성, 인디, 어쿠스틱, 새벽]","[쓸쓸함, 쓸쓸하게, 센치, 산책, 힐링, 혼자, 집중, 그리움, 조용한, 여유]",0,0
9,[힐링],"[에너지, 여행, 인디, 드라이브]","[연주음악, 뉴에이지, 생각, 독서, 피아노, 여유로운, 책, 편안한, 연주곡, 여유]",0,0
